In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(model="openai/gpt-oss-120b")

In [ ]:
from langgraph.graph import START, END, StateGraph
from typing import TypedDict, Annotated, List
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage
from langchain_core.messages import (
    SystemMessage,
    HumanMessage,
    AIMessage,
    ToolMessage
)


In [ ]:
from typing import Dict

def get_weather(location: str) -> Dict:
    data = {
        "bangalore": {"temp": 28, "condition": "Cloudy"},
        "chennai": {"temp": 35, "condition": "Sunny"},
        "delhi": {"temp": 22, "condition": "Foggy"},
    }

    return data.get(
        location.lower(),
        {"temp": 30, "condition": "Mysterious"}
    )


In [ ]:
from langchain.tools import Tool

weather_tool = Tool(
    name="get_weather",
    func=get_weather,
    description="Get weather and temperature for a given location"
)


In [ ]:
from langgraph.prebuilt import create_react_agent
from langchain_groq import ChatGroq

llm = ChatGroq(model="openai/gpt-oss-120b")

weather_agent = create_react_agent(
    llm=llm,
    tools=[weather_tool],
    system_prompt="""
You are a weather reporter.
You must:
1. Call the weather tool
2. Get temperature and condition
3. Describe the weather in a funny and engaging way
"""
)


In [ ]:
from langgraph_supervisor import create_supervisor

supervisor = create_supervisor(
    llm=llm,
    agents={
        "weather": weather_agent
    },
    system_prompt="""
You are a supervisor agent.
Your job:
- Understand the user's request
- Delegate weather-related tasks to the weather agent
- Return a final, engaging weather report
"""
)


In [ ]:
result = supervisor.invoke({
    "messages": [
        ("human", "Give me today's weather report for Bangalore")
    ]
})

print(result["messages"][-1].content)
